#### DS requests results via request/response cycle

A user can send a request to the domain owner to access a resource.

- The user can send a request by calling the `.request` method on the pointer. 
- A `reason` needs to be passed on a parameter while requesting access to the data.

#### The user selects a dataset and perform some query

- The user selects a network and domain
- The logs into the domain and selects a dataset
- The user perform a query on the selected dataset pointer

In [ ]:
import syft as sy

In [5]:
# Let's list the available dataset
sy.datasets

,Id,Name,Tags,Assets,Description,Domain,Network,Usage,Added On
0,b5b04506a0a0405f93ea0e2c0263ced6,Diabetes Dataset,"[Health, Classification, Dicom]","[""Images""] -> Tensor; [""Labels""] -> Tensor",A large set of high-resolution retina images,California Healthcare Foundation,WHO,102,Jan 20 2021
1,246a5e7835324a08b2883037257612dd,Canada Commodities Dataset,"[Commodities, Canada, Trade]","[""ca-feb2021""] -> DataFrame",Commodity Trade Dataset,Canada Domain,United Nations,40,Mar 11 2021
2,92c0719f990d4bcf85ccd039e2e034ba,Italy Commodities Dataset,"[Commodities, Italy, Trade]","[""it-feb2021""] -> DataFrame",Commodity Trade Dataset,Italy Domain,United Nations,23,Mar 20 2021
3,20b93adea1504538a7d03d7c3f248c20,Netherlands Commodities Dataset,"[Commodities, Netherlands, Trade]","[""ne-feb2021""] -> DataFrame",Commodity Trade Dataset,Netherland Domain,United Nations,20,Apr 12 2021
4,fc650f14f6f7454b87d3ccd345c437b5,Pnuemonia Dataset,"[Health, Pneumonia, X-Ray]","[""X-Ray-Images""] -> Tensor; [""labels""] -> Tensor",Chest X-Ray images. All provided images are in...,RSNA,WHO,334,Jan 20 2021


In [7]:
# We want to access the `Pneumonia Dataset`, let's connect to the RSNA domain
# Let's list the RSNA available networks
sy.networks

,Id,Name,Hosted Domains,Hosted Datasets,Description,Tags,Url
0,33c077db2e6249b49713bf7f53708524,United Nations,4,6,The UN hosts data related to the commodity and...,"[Commodities, Census, Health]",https://un.openmined.org
1,cc2044861d534a6c98f0a669175af9bf,World Health Organisation,3,5,WHO hosts data related to health sector of dif...,"[Virology, Cancer, Health]",https://who.openmined.org
2,3981dfa140c445cf90e23e6ef4b8fe9c,International Space Station,2,4,ISS hosts data related to the topography of di...,"[Exoplanets, Extra-Terrestrial]",https://iss.openmined.org


In [14]:
# Let's select the `WHO` network and list the available domains on the `WHO Network`.
who_network = sy.networks[1]
who_network.domains

,Id,Name,Hosted Datasets,Description,Tags
0,02631a42192942e08ea0ab10598f2c79,California Healthcare Foundation,1,Health care systems,"[Clinical Data, Healthcare]"
1,e1640cc4af70422da1d60300724b1ee3,RSNA,1,Radiological Image Datasets,"[Dicom, Radiology, Health]"


In [ ]:
# Let's select the `RSNA domain`
rsna_domain = who_network["e1640cc4af70422da1d60300724b1ee3"]

# Let's login into the rsna domain
rsna_domain_client = rsna_domain.login(email="sheldon@caltech.edu", password="bazinga")

# Let's select the pnuemonia dataset
pnuemonia_dataset = rsna_domain_client["fc650f14f6f7454b87d3ccd345c437b5"]

In [19]:
# Let's see the dataset
pnuemonia_dataset


Name: Pnuemonia Detection and Locationzation Dataset
Description: Chest X-Ray images. All provided images are in DICOM format.



,Asset Key,Type,Shape
0,[X-Ray-Images],Tensor,"(40000, 7)"
1,[labels],Tensor,"(40000, 5)"


In [ ]:
# Let's select the lable tensors
label_ptr = pnuemonia_dataset["labels"]

# Let's calculate the unique labels in the dataset
unique_labels = label_ptr[:,0].unique()

#### The user fetches the results of their query

- The user can perform a `.get` operation to download the data of the variable locally.
- If a user tries to access a variable without publishing its results or without requesting it, then they receive a 403.
- If a user has requested a resource, then its denied by the DO, then the user receives a 403 on performing a get operation on the resource.

In [43]:
number_of_unique_labels = unique_labels.shape
# Let's access the labels
number_of_unique_labels.get()


    PermissionDenied:
        You don't have authorization to perform the `.get` operation.
        You need to either `request` the results or `publish` the results.



In [16]:
# Let's request the results from the Domain Owner
number_of_unique_labels.request(reason="Know the number of unique labels in the dataset.")


    Your request has been submitted to the domain. Your request id is: a825a64be418430da8869a1f4542a6c0
    You can check the status of your requests via `.requests`.



#### The user views the request logs

- The user can list all the logs of all the requests send by them to a domain. **[P1]**

    Following properties are visible to the user w.r.t to the logs (for data requests):

    - Request Id (Unique id of the request)
    - Request Date (Datetime on which the request was submitted. The datetime/timestamp are shown in UTC)
    - Reason (The reason submitted to access the resource by requester)
    - Result Id (The unique id of the reasource being requested)
    - State (State of the request - Approved/Declined/Pending)
    - Reviewer Comments (Comment provided by the reqeuest reviewer (DO) during request approval/deny)

- The user can filter through the logs via ID and Status. **[P2]**

In [12]:
# Let's check the status of our request logs
# A user can see only the request logs with state == `Pending`.
rsna_domain.requests

You have 1 pending request


,Request Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,3ca9694c8e5d4214a1ed8025a1391c8c,Sep 29 2021 11:31AM,Know the number of unique labels in the dataset.,315411ed283042ee81c78737d2010ca5,Pending,-


In [13]:
# If we want to see all the requests that are submitted to the domain,
rsna_domain.requests.all()

,Request Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,3ca9694c8e5d4214a1ed8025a1391c8c,Sep 29 2021 11:31AM,Know the number of unique labels in the dataset.,315411ed283042ee81c78737d2010ca5,Pending,-
1,6ee8bbea539e4088ad4c23a7a123cfbb,Sep 19 2021 01:31AM,Get the labels in the dataset.,c1498711dd5f4a1682bfe2c8bba8830f,Denied,Access to raw labels is not allowed


```
Some time has passed, let' check if our requests are approved or not.
```

In [18]:
# Let's check again for pending requests first....
rsna_domain.requests

There are no pending reqeuests.


In [14]:
# Let's check all the submitted requests
rsna_domain.requests.all()

,Request Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,3ca9694c8e5d4214a1ed8025a1391c8c,Sep 29 2021 11:31AM,Know the number of unique labels in the dataset.,315411ed283042ee81c78737d2010ca5,Approved,Looks good.
1,6ee8bbea539e4088ad4c23a7a123cfbb,Sep 19 2021 01:31AM,Get the labels in the dataset.,c1498711dd5f4a1682bfe2c8bba8830f,Denied,Access to raw labels is not allowed


In [90]:
# Great our requests are approved, let's get the information
unique_labels = number_of_unique_labels.get()
print(f"Unique Labels: {len(unique_labels)}")

Unique Labels: 2


In [16]:
# Filtering requests logs

# via Id (Get the request with the given request id)
rsna_domain.requests.filter(id="3ca9694c8e5d4214a1ed8025a1391c8c")

# or via Status (List all the logs with given status)
rsna_domain.requests.filter(status="Approved")

,Request Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,3ca9694c8e5d4214a1ed8025a1391c8c,Sep 29 2021 11:31AM,Know the number of unique labels in the dataset.,315411ed283042ee81c78737d2010ca5,Approved,Looks good.


#### Dummy Data

In [1]:
import pandas as pd
from enum import Enum
import uuid
import torch
import datetime
import json
import numpy as np


class bcolors(Enum):
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

In [2]:
all_datasets = [
    {
        "Id": uuid.uuid4().hex,
        "Name": "Diabetes Dataset",
        "Tags": ["Health", "Classification", "Dicom"],
        "Assets": '''["Images"] -> Tensor; ["Labels"] -> Tensor''',
        "Description": "A large set of high-resolution retina images",
        "Domain": "California Healthcare Foundation",
        "Network": "WHO",
        "Usage": 102,
        "Added On": datetime.datetime.now().replace(month=1).strftime("%b %d %Y")
    },
    {
        "Id": uuid.uuid4().hex,
        "Name": "Canada Commodities Dataset",
        "Tags": ["Commodities", "Canada", "Trade"],
        "Assets": '''["ca-feb2021"] -> DataFrame''',
        "Description": "Commodity Trade Dataset",
        "Domain": "Canada Domain",
        "Network": "United Nations",
        "Usage": 40,
        "Added On": datetime.datetime.now().replace(month=3, day=11).strftime("%b %d %Y")
    },
    {
        "Id": uuid.uuid4().hex,
        "Name": "Italy Commodities Dataset",
        "Tags": ["Commodities", "Italy", "Trade"],
        "Assets": '''["it-feb2021"] -> DataFrame''',
        "Description": "Commodity Trade Dataset",
        "Domain": "Italy Domain",
        "Network": "United Nations",
        "Usage": 23,
        "Added On": datetime.datetime.now().replace(month=3).strftime("%b %d %Y")
    },
    {
        "Id": uuid.uuid4().hex,
        "Name": "Netherlands Commodities Dataset",
        "Tags": ["Commodities", "Netherlands", "Trade"],
        "Assets": '''["ne-feb2021"] -> DataFrame''',
        "Description": "Commodity Trade Dataset",
        "Domain": "Netherland Domain",
        "Network": "United Nations",
        "Usage": 20,
        "Added On": datetime.datetime.now().replace(month=4, day=12).strftime("%b %d %Y")
    },
    {
        "Id": uuid.uuid4().hex,
        "Name": "Pnuemonia Dataset",
        "Tags": ["Health", "Pneumonia", "X-Ray"],
        "Assets": '''["X-Ray-Images"] -> Tensor;  ["labels"] -> Tensor''',
        "Description": "Chest X-Ray images. All provided images are in DICOM format.",
        "Domain": "RSNA",
        "Network": "WHO",
        "Usage": 334,
        "Added On": datetime.datetime.now().replace(month=1).strftime("%b %d %Y")
    },
]


all_datasets_df = pd.DataFrame(all_datasets)

In [3]:
# Print available networks

available_networks = [
    {
        "Id": f"{uuid.uuid4().hex}",
        "Name": "United Nations",
        "Hosted Domains": 4,
        "Hosted Datasets": 6,
        "Description": "The UN hosts data related to the commodity and Census data.",
        "Tags": ["Commodities", "Census", "Health"],
        "Url": "https://un.openmined.org",
    },
    {
        "Id": f"{uuid.uuid4().hex}",
        "Name": "World Health Organisation",
        "Hosted Domains": 3,
        "Hosted Datasets": 5,
        "Description": "WHO hosts data related to health sector of different parts of the worlds.",
        "Tags": ["Virology", "Cancer", "Health"],
        "Url": "https://who.openmined.org",
    },
    {
        "Id": f"{uuid.uuid4().hex}",
        "Name": "International Space Station",
        "Hosted Domains": 2,
        "Hosted Datasets": 4,
        "Description": "ISS hosts data related to the topography of different exoplanets.",
        "Tags": ["Exoplanets", "Extra-Terrestrial"],
        "Url": "https://iss.openmined.org",
    },
]
networks_df = pd.DataFrame(available_networks)

In [4]:
who_domains = [
    {
        "Id": f"{uuid.uuid4().hex}",
        "Name": "California Healthcare Foundation",
        "Hosted Datasets": 1,
        "Description": "Health care systems",
        "Tags": ["Clinical Data", "Healthcare"],
    },
    {
        "Id": f"{uuid.uuid4().hex}",
        "Name": "RSNA",
        "Hosted Datasets": 1,
        "Description": "Radiological Image Datasets",
        "Tags": ["Dicom", "Radiology", "Health"],
    },
]
who_domains_df = pd.DataFrame(who_domains)

In [5]:
pneumonia_dataset = [
    {
        "Asset Key": "[X-Ray-Images]",
        "Type": "Tensor",
        "Shape": "(40000, 7)"
    },
    {
        "Asset Key": '[labels]',
        "Type": "Tensor",
        "Shape": "(40000, 5)"
    },
]
print("""
Name: Pnuemonia Detection and Locationzation Dataset
Description: Chest X-Ray images. All provided images are in DICOM format.
""")
pneumonia_dataset_df = pd.DataFrame(pneumonia_dataset)


Name: Pnuemonia Detection and Locationzation Dataset
Description: Chest X-Ray images. All provided images are in DICOM format.



In [6]:
labels_data = np.random.randint(0, 2, size=(40000, 5))[:, 0]
label_tensors = torch.Tensor(labels_data)

In [7]:
authorization_error = f"""
    {bcolors.FAIL.value}PermissionDenied:{bcolors.ENDC.value}
        You don't have authorization to perform the `.get` operation.
        You need to either `request` the results or `publish` the results.
"""

print(authorization_error)


    PermissionDenied:
        You don't have authorization to perform the `.get` operation.
        You need to either `request` the results or `publish` the results.



In [8]:
request_uuid = uuid.uuid4().hex
request_submission = f"""
    Your request has been submitted to the domain. Your request id is: {bcolors.BOLD.value}{request_uuid}
    {bcolors.ENDC.value}You can check the status of your requests via `.requests`.
"""
print(request_submission)


    Your request has been submitted to the domain. Your request id is: 8b0f17f8ef284e09b1afb75443b1f4a7
    You can check the status of your requests via `.requests`.



In [9]:
requests_data = [
    {
        "Request Id": uuid.uuid4().hex,
        "Request Date": datetime.datetime.now().strftime("%b %d %Y %I:%M%p"),
        "Reason": "Know the number of unique labels in the dataset.",
        "Result Id": uuid.uuid4().hex,
        "State": "Pending",
        "Reviewer Comments": "-",
    },
    {
        "Request Id": uuid.uuid4().hex,
        "Request Date": datetime.datetime.now().replace(day=19, hour=1).strftime("%b %d %Y %I:%M%p"),
        "Reason": "Get the labels in the dataset.",
        "Result Id": uuid.uuid4().hex,
        "State": "Denied",
        "Reviewer Comments": "Access to raw labels is not allowed",
    }
]

requests_data_df = pd.DataFrame(requests_data)

In [10]:
approved_requests_data_df = requests_data_df.copy()
approved_requests_data_df["State"][0] = "Approved"
approved_requests_data_df["Reviewer Comments"][0] = "Looks good."

In [11]:
filtered_request_logs = approved_requests_data_df[:1]